#作成したCNNを用いて動画認識を行う
参考資料
https://note.nkmk.me/python-opencv-videocapture-file-camera/


In [0]:
#driveとの連携
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/School Project/System Design Project/recognize_num/

/content/drive/My Drive/School Project/System Design Project/recognize_num


In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from chainer import Sequential
import chainer.functions as F
import chainer.links as L
import chainer
from google.colab.patches import cv2_imshow
from pathlib import Path

In [0]:
#動画ファイルの読み込み
cap = cv2.VideoCapture('/content/drive/My Drive/School Project/System Design Project/recognize_num/movie/num.mp4')
print(cap.isOpened())#読み込めたかどうか


True


In [0]:
#幅
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#高さ
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# FPS
print(cap.get(cv2.CAP_PROP_FPS))
# 層フレーム数
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#動画秒数
print(cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS))

720.0
480.0
30.0
265.0
8.833333333333334


In [0]:
#動画の現在のフレーム位置
print(cap.get(cv2.CAP_PROP_POS_FRAMES))

0.0


In [0]:
#学習データの読み込み
nnfilepass = '/content/gdrive/My Drive/School Project/System Design Project/recognize_num/num_cnn.net'
loaded_net = Sequential(L.Linear(784, 1000),F.relu,L.Linear(1000, 1000),F.relu,L.Linear(1000, 10))
chainer.serializers.load_npz('num_pic.net', loaded_net)

In [0]:
#画像をデータにして
output_dir = Path("output_cnn")
output_dir.mkdir(exist_ok=True)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # フレーム数を取得する。
n_digits = len(str(n_frames))  

In [0]:
#


while True: 
    ret, frame = cap.read()#1フレーム読み込み
    if ret == False:#フレームが読み込めなかったら出る
        break
    
    f_frame = frame.astype(np.uint8)#int変換
    bulr_img =cv2.GaussianBlur(f_frame,(15,15), 0)
    gray_img = cv2.cvtColor(bulr_img, cv2.COLOR_RGB2GRAY)
    plt.gray()
    threshold_value=100
    ret, thresh_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)
    fc_img,contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    point_list = list()
    for i in range(0, len(contours)):
        if len(contours[i]) > 0:
            if hierarchy[0][i][3] != 0:
                continue
            rect = contours[i]
            x, y, w, h = cv2.boundingRect(rect)
            point_list.append([x,y,w,h])
    imglist = list()
    for point in point_list:
        imglist.append(gray_img[point[1]:point[1]+point[3],point[0]:point[0]+point[2]])
    resize_imglist = list()
    for i in range(len(imglist)):
        #print(np.shape(imglist[i]))
        resize_imglist.append(cv2.resize(imglist[i], (28,28),0,0 ,cv2.INTER_NEAREST))
    rev_imglist=list()
    for i in range(len(resize_imglist)):
        rev_imglist.append(cv2.bitwise_not(resize_imglist[i]))
    flat_imglist=list()
    for i in range(len(rev_imglist)):
        flat_imglist.append(rev_imglist[i].flatten())

    arr = np.array(flat_imglist, dtype=np.float32)
    
    if np.shape(arr)[0] != 0 :
        with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
            prolist = loaded_net(arr)
        copy_img = f_frame.copy()
        con = 0
        for point in point_list :
            cv2.putText(copy_img, str(np.argmax(prolist[con,:].array)), (point[0], point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 0), 2, 8)
            cv2.rectangle(copy_img, (point[0], point[1]), (point[0] + point[2], point[1] + point[3]), (0, 100, 0), 2)
            con += 1     

        #copy_imgから画像を作る
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  #フレーム番号
        save_path = output_dir / f"frame_{frame_no:0{n_digits}d}.png"#パス
        cv2.imwrite(str(save_path), copy_img)#セーブ

    else:#認識に失敗したとき
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  #フレーム番号
        save_path = output_dir / f"frame_{frame_no:0{n_digits}d}.png"#パス
        cv2.imwrite(str(save_path), f_frame)#セーブ


cap.release()  

<Figure size 432x288 with 0 Axes>

In [0]:
#画像ファイルを読み込み動画にして保存する
img_paths = Path("output") / "frame_%03d.png"

# VideoCapture を作成する。
rec_video = cv2.VideoCapture(str(img_paths))
width = int(rec_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(rec_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 15
print(f"width: {width}, height: {height}, fps: {fps}")

# VideoWriter を作成する。
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
writer = cv2.VideoWriter("output.avi", fourcc, fps, (width, height))

while True:
    # 1フレームずつ取得する。
    ret, frame = rec_video.read()
    if not ret:
        break  # フレームの取得に失敗または動画の末尾に到達した場合

    writer.write(frame)  # フレームを書き込む。

writer.release()
rec_video.release()

width: 720, height: 480, fps: 15


# 結果
よくはない。境界部分に数字が来ると認識できない。  
たぶん境界値分析を階層構造でやっているので階層がずれるんだと思う。